In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [ ]:
website_url = 'https://countrymusichalloffame.org/hall-of-fame/members/'
response = requests.get(website_url)

response.status_code

In [ ]:
print(type(response))
response.content

In [ ]:
country_star = BeautifulSoup(response.content, 'lxml')
print(country_star.title)

In [ ]:
print(country_star)

In [ ]:
artist_names = country_star.find_all('h3')
artist_names

In [ ]:
type(artist_names)

In [ ]:
artist_list = [ ]
for artist in artist_names:
    artist_list.append(artist.text)

In [ ]:
artist_list = [artist.text for artist in artist_names]

In [ ]:
artist_list

In [ ]:
type(artist_names)

In [ ]:
#Add 2nd page with same code

### Question 2

In [ ]:
year_inducted_soup = country_star.find_all('p')

In [ ]:
year_inducted_soup = country_star.find_all('div', attrs = {'class': 'vertical-card_content--copy'})

In [ ]:
len(year_inducted_soup)

In [ ]:
year_inducted_soup[0].text

In [ ]:
years_inducted = [int(re.findall('\d+', year_str.text)[0]) for year_str in year_inducted_soup
                     if re.match("\s+Inducted\s\d+\s+", year_str.text)]

In [ ]:
years_inducted

In [ ]:
years =[int(year_inducted.text.strip().replace('Inducted ',"")) for year_inducted in year_inducted_soup]

In [ ]:
years

In [ ]:
years = []
#iteration = 0
for year_inducted in year_inducted_soup:
#   print
    years.append(int(year_inducted.text.strip().replace('Inducted ',"")))
years

In [ ]:
new_years = []
for i in range(len(years)):
    new_years.append(years[i] - 2000)

In [ ]:
new_years

### Question 3 Combine list

In [ ]:
artist_list

In [ ]:
years_inducted

In [ ]:
hof_df=pd.DataFrame(
    {
        "artist":artist_list,
        "years_inducted":years_inducted
    }
)

In [ ]:
df=pd.DataFrame()
df['artist'] = artist_list

In [ ]:
df

In [ ]:
hof_df

In [ ]:
list(zip(artist_list,years_inducted))

In [ ]:
results_together = pd.DataFrame(list(zip(artist_list, years_inducted)), columns = ['Artist', 'Year'])
results_together

### Question 4

In [ ]:
test_list = [1, 2, 3, 4]
test_list.extend([5, 6, 7, 8])
test_list

In [ ]:
all_artist = []
all_years_inducted = []
for page in range(1, 17):
    if page == 1:
        hof_url = "https://countrymusichalloffame.org/hall-of-fame/members/"
    else:
        hof_url = f"https://countrymusichalloffame.org/hall-of-fame/members/page/{page}"
    response = requests.get(hof_url)
    soup = BeautifulSoup(response.content, 'lxml')
    
    artist_page = soup.find_all('h3')
    artist_list_page = [artist.text for artist in artist_page]
    
    years_inducted_page =soup.find_all('p')
    years_inducted_list_page = [int(re.findall("\d+", year_str.text)[0]) for year_str in years_inducted_page
                               if re.match("\s+Inducted\s\d+\s+", year_str.text)]
    
    all_artist.extend(artist_list_page)
    all_years_inducted.extend(years_inducted_list_page)

In [ ]:
#all_artist

In [ ]:
hof_df_all = pd.DataFrame(
    {
        "artist": all_artist,
        "year_inducted": all_years_inducted
    }
)
hof_df_all.head()

In [ ]:
hof_df_all['name_len'] = hof_df_all['artist'].apply(lambda x: len(x))

In [ ]:
hof_df_all.info()

In [ ]:
#hof_df_all.iloc[50:100]

### Question 5

In [ ]:
hof_df_all.year_inducted.value_counts().sort_index().plot.bar()

In [ ]:
hof_sorted = hof_df_all.year_inducted.value_counts().sort_index().reset_index()

In [ ]:
#hof_sorted

In [ ]:
plt.figure(figsize = (18,6))
sns.barplot(x = hof_sorted['index'], y = hof_sorted['year_inducted'])

In [ ]:
plt.plot()

### Bonus Question

In [ ]:
test_string = "Birth: September 15, 1903 - Death: November 23, 1992 Birthplace: Maynardville, Tennessee"

In [ ]:
re.findall("Birth: (\w+)", test_string)

In [ ]:
re.sub("[a-z]", "", test_string)

In [ ]:
roy_url = 'https://countrymusichalloffame.org/artist/roy-acuff/'
roy_response = requests.get(roy_url)
roy_soup = BeautifulSoup(roy_response.content, 'lxml')

In [ ]:
info_style = 'margin-top: 20px;color: #50565A;font-size: 14px;font-weight: 500;letter-spacing: 1.5px;line-height: 25px; text-transform: uppercase;'
roy_info = roy_soup.find_all('p', attrs = {'style': info_style})

In [ ]:
info_style = 'margin-top: 20px;color: #50565A;font-size: 14px;font-weight: 500;letter-spacing: 1.5px;line-height: 25px; text-transform: uppercase;'
births = []
deaths = []
birthplaces = []
for artist_name in hof_df_all["artist"].to_list():
    artist_page_name = '-'.join(re.findall("\w+", artist_name)).lower()
    print(artist_page_name)
    
    artist_url = f'https://countrymusichalloffame.org/artist/{artist_page_name}/'
    artist_response = requests.get(artist_url)
    soup = BeautifulSoup(artist_response.content, 'lxml')
    print(artist_url)
    
    artist_info = soup.find_all('p', attrs = {'style': info_style})
    if artist_info:
        birthdate = re.findall("Birth: (\w+\s\d+, \d+)\s", artist_info[0].text)
        deathdate = re.findall("Death: (\w+\s\d+, \d+)\s", artist_info[0].text)
        birthplace = re.findall("Birthplace: (.+)\s$", artist_info[0].text)
    else:
        birthdate = [""]
        deathdate = [""]
        birthplace = [""]
    if len(birthdate) == 0:
        birthdate = [""]
    if len(deathdate) == 0:
        deathdate = [""]
    if len(birthplace) == 0:
        birthplace = [""]
    print(artist_info)
    
    print(f"Birth Date: {birthdate}")
    
    print(f"Death Date: {deathdate}")
    
    print(f"Birthplace: {birthplace}")
    
    print("==========================")
    births.extend(birthdate)
    deaths.extend(deathdate)
    birthplaces.extend(birthplace)
    
   

In [ ]:
mbr=str(tables[0])

In [ ]:
re.findall(r"[^()0-9-]+", mbr)

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://countrymusichalloffame.org/hall-of-fame/members/"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# delete all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # extract

# get text
text = soup.body.get_text(separator= ' ')

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

print(text)

In [ ]:
ppl=str(text[0])

In [ ]:
re.findall(r"Roy", ppl)

In [ ]:
members = country_star.find_all('div', attrs = {'class': 'vertical-card_content--title'})
members 

In [ ]:
type(members)

In [ ]:
for members in members:
     print(members, end='\n'*2)

artist = []
for i in range(len(members)):
    name = members[i].get_text.replace('\n','')
artist.append(name)
print(artist)
type(members)